In [ ]:
import pandas as pd
import math
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [ ]:
df = pd.read_excel('/Users/liucheng/Desktop/Yuecheng_LIMS_QC_MiniProject/Bacterial cell bank_mock data.xlsx')
df.head()
# Import raw data  and display the first five rows
# Rename column for superscript
df = df.rename(columns={'Cell Viability (107)':'Cell Viability (10⁷)'})
df.head()

In [ ]:
# Define specifications for each test. In a pharmaceutical lab, specifications are defined in the documentation system and are regularly updated to ensure compliance with quality standards
spec_dict = {
    'Plasmid Retention Test (%)':{'type':'numeric','min':85},
        'Cell Viability (107)':{'type':'numeric','min':1.00},
       'Product Expression_Result':{'type':'categorical','allowed':['Pass']}
}
# Define a function to flag Pass/Fail
def flag_result(row, spec_dict):
    for test, rules in spec_dict.items():
        if test in row:
            if rules['type']=='numeric':
                value = row[test]
                min_val = rules.get('min', -math.inf)
                max_val = rules.get('max',math.inf)
                if not(min_val <= value<= max_val):
                    return 'Fail'
            elif rules['type']=='categorical':
                if row[test] not in rules['allowed']:
                    return 'Fail'
    return 'Pass'

# Add a column in excel called "Pass/Fail" check
df['Pass/Fail']= df.apply(lambda row:flag_result(row, spec_dict), axis=1)

#Save the QC testing results in a new excel file called Bacterial Cell Bank_qc_results.xlsx
excel_file ='/Users/liucheng/Desktop/Yuecheng_LIMS_QC_MiniProject/Bacterial Cell Bank_qc_results.xlsx'
df.to_excel(excel_file, index=False)

# I want to fill the cells with "Pass" in green, "Fail" in red
# Open the Excel with openpyxl to add colors
wb = load_workbook(excel_file)
ws = wb.active

# Define fills
fills = {
    'Pass': PatternFill(fill_type="solid", start_color="90EE90"),  # light green
    'Fail': PatternFill(fill_type="solid", start_color="FF7F7F")   # light red
}

# Highlight entire row if Fail
for row in range(2, ws.max_row + 1):  # skip header
    pass_fail_cell = ws.cell(row=row, column=df.columns.get_loc("Pass/Fail")+1)
    if pass_fail_cell.value == "Fail":
        for col in range(1, ws.max_column + 1):
            ws.cell(row=row, column=col).fill = fills["Fail"]
    else:
        ws.cell(row=row, column=df.columns.get_loc("Pass/Fail")+1).fill = fills["Pass"]
# Save final colored Excel
wb.save('/Users/liucheng/Desktop/Yuecheng_LIMS_QC_MiniProject/Bacterial Cell Bank_qc_results_colored.xlsx')